This notebook is intended to run the full LandTrendr Optimization (LTOP) workflow implemented in Python. This tool is intended to be used to select the 'optiumum' version of the LandTrendr change detection algorithm (Kennedy et al., 2010, 2018) for different areas on a landscape. 

#### Notes
- You need to authenticate the run with a specific GEE account. In the import params statement its going to trigger the ee.Authenticate() protocal which will prompt you to a browser page to authenticate using your GEE account. 
- Subsequent scripts have just the ee.Initialize protocal because you should have already authenicated and this should only happen once. 
- Right now this is going to import the params from a separate .py file and treat those as imports. It may actually be easier/more straight forward if we're going to stick with a Jupyter Notebook to just put these directly into a cell here in the Jupyter Notebook. 
- If you import a module at the top and then change something in the exe script you will need to restart or otherwise delete variables because that change won't be reflected otherwise. 

In [7]:
import ee 
import params
import time 
import pandas as pd 
import ltop 
import importlib
importlib.reload(ltop)
import lt_params
importlib.reload(lt_params)
from run_SNIC_01 import RunSNIC
import run_kMeans_02_1 as kmeans_1
import run_kMeans_02_2 as kmeans_2
import abstract_sampling_03 as ab_img
import abstract_imager_04 as run_lt_pts


There is potentially an issue here where running a cell for a large area will result in a problem with the cell hanging. This may be an issue particuarly if you're running this thing in a browser. Its also possible that it will just start a job on the GEE server and then finish running here in the notebook. 

In order to check if a task is done we can get the task status and check to see if its done. Define a little function that can be recycled below for that purpose. 

In [ ]:
def check_task_status(task_dict): 
    '''
    Input to this function should be a dictionary that is formatted 
    like the output of task.status()
    '''
    task_id = task_dict['id']
    #for some reason GEE defaults this to a list with a dictioanary as its only item
    task_status = ee.data.getTaskStatus(task_id)[0]
    
    return task_status['state']

In [ ]:
#run the first SNIC step 

snic = RunSNIC(params)

status1,status2 = snic.generate_tasks()


In the next step we run the kmeans algorithm which will automatically grab the snic outputs to do that process. This is not exposed to the user and assumes that you have specified the child and root directories where you want things to go in the params.py file. The following code block will check if the snic process is done and then when it determines that process has concluded it will execute the kmeans step. 

In [ ]:

#note that if you want to just run the kmeans process without having run snic 
#uncomment the following line of code
km_status = kmeans_1.generate_tasks(params.params)
while True:
    try: 
        ts_1 = check_task_status(status1) 
        ts_2 = check_task_status(status2) 
        
        if (ts_1 == 'COMPLETED') & (ts_2 == 'COMPLETED'): 
            print('The previous task is complete')
            km_status = kmeans_1.generate_tasks(params.params)
            break
        elif (ts_1 == 'FAILED') | (ts_1 == 'CANCELLED'): 
            print('The first task failed')
            break
        elif (ts_2 == 'FAILED') | (ts_2 == 'CANCELLED'): 
            print('The second task failed')
            break 
    except NameError: 
        print('You did not run the snic step so there is no status to check')
        break
    

In the next step we just take the kmeans output and do a stratified random sample to get one point for each cluster id in the kmeans output image. Like the previous step, this one will also check to see if the output is done before executing. 

In [ ]:
km_pts_status = kmeans_2.generate_tasks(params.params)


The next step is to create the abstract images. Previously to the Python implementation, these were created from a CSV which was generated in GEE and then pulled down to a local machine. The actual images were constructed in Numpy and then re-uploaded to GEE. This is a pretty inefficient process and therefore we are moving it to a GEE-assets generation type process. This is based on some code that Jack Kilbride wrote to replace the Numpy scheme and is still in testing as of 10/6/2022. 

In [ ]:
ab_imgs_status = ab_img.create_abstract_imgs(params.params)

In [6]:
#test to see if we can just make the runParams into something else
df = pd.DataFrame.from_records(lt_params.runParams)#.reset_index()#,index=range(len(lt_params.runParams)))

# df = df.head(10)
df['timeseries'] = None
df['timeseries'] = ee.ImageCollection([])
output = df.to_dict(orient='records')

ls = [x for x in output]
ls[0]


# df = df.head(1)
# def something(x): 
#     print(x.to_frame().T.to_dict(orient='dict'))
# df.apply(something)

# test = df.iloc[0].to_frame().T
# test.timeseries
# df = df.apply(lambda x: print(pd.DataFrame(x).to_dict(orient='dict')))

# def something(x): 
#     print(x)
#     return None
# test = df.head(10)
# test = df.apply(lambda x: print(x.iloc[:,0]))

{'timeseries': <ee.imagecollection.ImageCollection at 0x7f381bfacb90>,
 'maxSegments': 6,
 'spikeThreshold': 0.75,
 'vertexCountOvershoot': 3,
 'preventOneYearRecovery': True,
 'recoveryThreshold': 0.25,
 'pvalThrec': 0.05,
 'bestModelProportion': 0.75,
 'minObservationsNeeded': 6,
 'pvalThreshold': nan}

Next we run landtrendr on the abstract image points, using the indices generated in the previous step as inputs. The scripts for the 04 step should be prepped to handle assets as inputs and will expect an imageCollection of abstract images as well as the points that were generated in the previous step that show where the abstract image pixels are located (centroids). 

In [8]:
lt_pt_status = run_lt_pts.run_LT_abstract_imgs(params.params)

Count:  7

